    If you want to use GPU...

In [1]:
gpu = "1" #iF YOU WANT TO USE GPU
# gpu = None 

import os
import torch
#----------------------------------------------------------------
# Selecting the device
#----------------------------------------------------------------
if gpu:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu #aca se pone nuemro de grafica libre

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'Using GPU: {torch.cuda.get_device_name()}')
    print('CUDA Visible devices:',os.getenv('CUDA_VISIBLE_DEVICES'))
else:
    device = torch.device('cpu')
    print("Failed to find GPU, using CPU instead.")


Using GPU: NVIDIA GeForce RTX 3080
CUDA Visible devices: 1


# Imports

In [3]:
# !pip install tensorboard

In [2]:
#torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import tensorboard as tb
from torchvision import datasets, transforms

#others
from tqdm.notebook import tqdm

# Simple Model

This will be our reference model for all the examples

In [4]:
# Simple CNN
class simple_CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(simple_CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

# Example No.1

In [9]:
# Hyperparameters
lr = 0.001
in_channels = 1
num_classes = 10
batch_size = 64
num_epochs = 3

# Load Data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform = transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

#Initialize network
model = simple_CNN(in_channels=in_channels, num_classes=num_classes)
model.to(device) #move model to device

#Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

"""
TENSORBOARD WRITER 
"""
writer = SummaryWriter(f'runs/MNIST/using_tensorboard')
step = 0

for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
        
        #Calculate running trining accuracy
        _, predictions = scores.max(1) # [[0,1],[1,0],[0,1],[]]
        num_correct = (predictions == targets).sum() #TP+TN
        running_train_acc = float(num_correct)/float(data.shape[0]) #ACC

        # tensorboard
        writer.add_scalar('Training loss', loss, global_step = step)
        writer.add_scalar('Training Accurcy', running_train_acc, global_step = step)
        step += 1 #step is the number of batches
        
       

# Example No.2

In [11]:
# Hyperparameters
in_channels = 1
num_classes = 10
num_epochs = 3
batch_sizes = [64,128,1024]
learning_rates = [0.01, 0.001, 0.0001]


# Load Data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform = transforms.ToTensor(), download=True)

for batch_size in tqdm(batch_sizes):
    for lr in learning_rates:
        step = 0        
        
        #dataloader
        train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        
        #Initialize network
        model = simple_CNN(in_channels=in_channels, num_classes=num_classes)
        model.to(device) #move model to device
        
        #Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        
        #TENSORBOARD WRITER 
        writer = SummaryWriter(f'runs/MNIST/batchSize_{batch_size}_lr_{lr}')    
        
        for epoch in range(num_epochs):
            for batch_idx, (data, targets) in enumerate(train_loader):
                # Get data to cuda if possible
                data = data.to(device=device)
                targets = targets.to(device=device)

                # forward
                scores = model(data)
                loss = criterion(scores, targets)

                # backward
                optimizer.zero_grad()
                loss.backward()

                # gradient descent or adam step
                optimizer.step()
                
                #Calculate running trining accuracy
                _, predictions = scores.max(1)
                num_correct = (predictions == targets).sum()
                running_train_acc = float(num_correct)/float(data.shape[0])

                # tensorboard
                writer.add_scalar('Training loss', loss, global_step = step)
                writer.add_scalar('Training Accurcy', running_train_acc, global_step = step)
                step += 1 #step is the number of batches

  0%|          | 0/3 [00:00<?, ?it/s]

# Example No.3
Using ``add_hparams``

In [12]:
# Hyperparameters
in_channels = 1
num_classes = 10
num_epochs = 3
batch_sizes = [128,1024]
learning_rates = [0.01, 0.001, 0.0001]


# Load Data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform = transforms.ToTensor(), download=True)

for batch_size in batch_sizes:
    for lr in tqdm(learning_rates):
        step = 0        
        
        #dataloader
        train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        
        #Initialize network
        model = simple_CNN(in_channels=in_channels, num_classes=num_classes)
        model.to(device) #move model to device
        
        #Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        
        #TENSORBOARD WRITER 
        writer = SummaryWriter(f'runs/MNIST/batchSize_{batch_size}_lr_{lr}')    
        
        for epoch in range(num_epochs):
            accuracies, losses = [], []
            for batch_idx, (data, targets) in enumerate(train_loader):
                # Get data to cuda if possible
                data = data.to(device=device)
                targets = targets.to(device=device)

                # forward
                scores = model(data)
                loss = criterion(scores, targets)
                losses.append(loss.item())
                
                # backward
                optimizer.zero_grad()
                loss.backward()

                # gradient descent or adam step
                optimizer.step()
                
                #Calculate running trining accuracy
                _, predictions = scores.max(1)
                num_correct = (predictions == targets).sum()
                running_train_acc = float(num_correct)/float(data.shape[0])
                accuracies.append(running_train_acc)
                # tensorboard
                writer.add_scalar('Training loss', loss, global_step = step)
                writer.add_scalar('Training Accurcy', running_train_acc, global_step = step)
                step += 1 #step is the number of batches
        writer.add_hparams(
            hparam_dict = {"lr": lr, "bsize": batch_size},
            metric_dict = {"Accuracy": sum(accuracies)/len(accuracies), 
                           "loss": sum(losses),
                          "time": time_sss}
        )

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

# Example 4

Saving weights

In [14]:
# Hyperparameters
in_channels = 1
num_classes = 10
num_epochs = 3
batch_sizes = [1024]
learning_rates = [0.01]


# Load Data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform = transforms.ToTensor(), download=True)

for batch_size in batch_sizes:
    for lr in tqdm(learning_rates):
        step = 0        
        
        #dataloader
        train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        
        #Initialize network
        model = simple_CNN(in_channels=in_channels, num_classes=num_classes)
        model.to(device) #move model to device
        
        #Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        
        #TENSORBOARD WRITER 
        writer = SummaryWriter(f'runs/MNIST/batchSize_{batch_size}_lr_{lr}_savingWeights')    
        
        for epoch in range(num_epochs):
            accuracies, losses = [], []
            for batch_idx, (data, targets) in enumerate(train_loader):
                # Get data to cuda if possible
                data = data.to(device=device)
                targets = targets.to(device=device)

                # forward
                scores = model(data)
                loss = criterion(scores, targets)
                losses.append(loss.item())
                
                # backward
                optimizer.zero_grad()
                loss.backward()

                # gradient descent or adam step
                optimizer.step()
                
                # histogram of weights
                writer.add_histogram("fc1", model.fc1.weight, global_step = step)
                writer.add_histogram("conv1", model.conv1.weight, global_step = step)
                writer.add_histogram("conv2", model.conv2.weight, global_step = step)
                #Calculate running trining accuracy
                _, predictions = scores.max(1)
                num_correct = (predictions == targets).sum()
                running_train_acc = float(num_correct)/float(data.shape[0])
                accuracies.append(running_train_acc)
                # tensorboard
                writer.add_scalar('Training loss', loss, global_step = step)
                writer.add_scalar('Training Accurcy', running_train_acc, global_step = step)
                step += 1 #step is the number of batches
        writer.add_hparams(
            hparam_dict = {"lr": lr, "bsize": batch_size},
            metric_dict = {"Accuracy": sum(accuracies)/len(accuracies), "loss": sum(losses)}
        )

  0%|          | 0/1 [00:00<?, ?it/s]

# Example 5.
Adding images

In [15]:
# Hyperparameters
in_channels = 1
num_classes = 10
num_epochs = 3
batch_sizes = [1024]
learning_rates = [0.01]


# Load Data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform = transforms.ToTensor(), download=True)
classes = ["0","1","2","3","4","5","6","7","8","9"]

for batch_size in batch_sizes:
    for lr in tqdm(learning_rates):
        step = 0        
        
        #dataloader
        train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
        
        #Initialize network
        model = simple_CNN(in_channels=in_channels, num_classes=num_classes)
        model.to(device) #move model to device
        
        #Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        
        #TENSORBOARD WRITER 
        writer = SummaryWriter(f'runs/MNIST/batchSize_{batch_size}_lr_{lr}_savingWeights')    
        
        for epoch in range(num_epochs):
            accuracies, losses = [], []
            for batch_idx, (data, targets) in enumerate(train_loader):
                # Get data to cuda if possible
                data = data.to(device=device)
                targets = targets.to(device=device)

                # forward
                scores = model(data)
                loss = criterion(scores, targets)
                losses.append(loss.item())
                
                # backward
                optimizer.zero_grad()
                loss.backward()

                # gradient descent or adam step
                optimizer.step()
                
                # histogram of weights
                writer.add_histogram("fc1", model.fc1.weight, global_step = step)
                writer.add_histogram("conv1", model.conv1.weight, global_step = step)
                writer.add_histogram("conv2", model.conv1.weight, global_step = step)
                #Calculate running trining accuracy
                _, predictions = scores.max(1)
                num_correct = (predictions == targets).sum()
                running_train_acc = float(num_correct)/float(data.shape[0])
                accuracies.append(running_train_acc)
                
                #Embeddings and images            
                img_grid = torchvision.utils.make_grid(data) #make grid of images
                writer.add_image("mnist_images", img_grid) #add image to tensorboard
                
                features = data.reshape(data.shape[0], -1) #embeddings  
                class_labels = [classes[label] for label in predictions]
                if batch_idx == 230: #only add embeddings once per epoch
                    writer.add_embedding(
                        features,
                        metadata=class_labels,
                        label_img=data,
                        global_step=batch_idx,
                    )
                
                # tensorboard
                writer.add_scalar('Training loss', loss, global_step = step)
                writer.add_scalar('Training Accurcy', running_train_acc, global_step = step)
                step += 1 #step is the number of batches
        writer.add_hparams(
            hparam_dict = {"lr": lr, "bsize": batch_size},
            metric_dict = {"Accuracy": sum(accuracies)/len(accuracies), "loss": sum(losses)}
        )

  0%|          | 0/1 [00:00<?, ?it/s]